In [1]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

In [2]:
#one prompt for all tipes of code
import tatqa_utils
import pandas as pd

In [3]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [13]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.llama31.db"))

In [14]:
df = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [15]:
from langchain_core.language_models.llms import LLM
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from typing import Any, Dict, List, Optional
import os
import uuid
import requests



class TritonLLM2(LLM):

    @property
    def _llm_type(self) -> str:
        return "tritonllm"
    
    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters."""
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": "CustomChatModel",
        }
        
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        #print("staring")
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        
        triton_url: str = os.environ["TRITON_LLM_ENDPOINT"]
        #trace_id: str = str(run_manager.parent_run_id)
        #span_id: str = str(run_manager.run_id)

        payload = {
            "text_input": prompt,
            "max_tokens": 1500,
            "temperature": 0.0,
            "stream": False
        }    

        ret = requests.post(
            triton_url,
            json=payload,
            #timeout=10,
            #headers= header_trace,
        )
        res = ret.json()
        query_response = res["text_output"]

        return query_response

In [16]:
os.environ["TRITON_LLM_ENDPOINT"] =  "http://172.22.214.120:28800/v2/models/llama-3.1-8b-instruct/generate"
llm = TritonLLM2()
llm.invoke("So?32")

'So?32\nSo?32 is a 32-bit operating system for the x86 architecture. It is a free and open-source operating system, and is based on the Linux kernel. So?32 is designed to be a lightweight and efficient operating system, with a focus on simplicity and ease of use.\nSo?32 was first released in 2003, and has since become a popular choice for users who want a simple and reliable operating system. It is known for its small footprint, low system requirements, and ease of use. So?32 is also highly customizable, with a wide range of software packages available for installation.\nSo?32 is not as widely used as some other operating systems, such as Windows or macOS, but it has a dedicated user base and is widely used in certain niches, such as in embedded systems and in some educational institutions.\nHere are some key features of So?32:\nSmall footprint: So?32 is a very small operating system, with a footprint of around 100 MB.\nLow system requirements: So?32 can run on very low-end hardware, m

In [17]:
def eval_res(res):
    good_type = 0
    for i in res:
       
       
        if (i["raw"]["answer_type"] == "arithmetic" and  i["answer2"] == "YES") or (i["raw"]["answer_type"] != "arithmetic" and  i["answer2"] == "NO"):
            good_type = good_type+1
        else:
            print("**", end="")        
            print(i["raw"]["answer_type"],"\t",i["answer2"],"\t", i["raw"]["answer_from"], "\t", i["question"])
   
    print(good_type, len(res), good_type / len(res))

In [18]:
res = []
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        #prompt2 = f"To answer the following question, is some arithmetic calculation required on data? QUESTION: {question} Please answer with one word: YES or NO."
        prompt2 = f"To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: {question}"
        answer2 = llm.invoke(prompt2)
        res.append({'raw': q, "question":question,  "prompt2": prompt2, "answer2": answer2})
eval_res(res)

**span 	 To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: What is the company paid on a cost-plus type contract? A) The company is paid a fixed price for the work. B) The company is paid a fixed price for the work plus a percentage of the cost. C) The company is paid a fixed price for the work plus a percentage of the cost plus a fee. D) The company is paid a fixed price for the work plus a percentage of the cost plus a fee plus a profit. E) The company is paid a fixed price for the work plus a percentage of the cost plus a fee plus a profit plus a bonus. 1. The correct answer is B. The company is paid a fixed price for the work plus a percentage of the cost. 2. The correct answer is C. The company is paid a fixed price for the work plus a percentage of the cost plus a fee. 3. The correct answer is D. The company is paid a fixed price for the work plus a percentage of the cost plus a fee plus a profit. 

In [19]:
os.environ["TRITON_LLM_ENDPOINT"] =  "http://172.22.214.120:28800/v2/models/llama-3.1-8b-instruct/generate"
llm = TritonLLM2()
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        #prompt2 = f"To answer the following question, is some arithmetic calculation required on data? QUESTION: {question} Please answer with one word: YES or NO."
        prompt2 = f"To answer the following question, is arithmetic calculation required on data? QUESTION: {question} Please answer with one word: YES or NO. Nothing else."
        answer2 = llm.invoke(prompt2)
        res.append({'raw': q, "question":question,  "prompt2": prompt2, "answer2": answer2})
eval_res(res)

**span 	 To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: What is the company paid on a cost-plus type contract? A) The company is paid a fixed price for the work. B) The company is paid a fixed price for the work plus a percentage of the cost. C) The company is paid a fixed price for the work plus a percentage of the cost plus a fee. D) The company is paid a fixed price for the work plus a percentage of the cost plus a fee plus a profit. E) The company is paid a fixed price for the work plus a percentage of the cost plus a fee plus a profit plus a bonus. 1. The correct answer is B. The company is paid a fixed price for the work plus a percentage of the cost. 2. The correct answer is C. The company is paid a fixed price for the work plus a percentage of the cost plus a fee. 3. The correct answer is D. The company is paid a fixed price for the work plus a percentage of the cost plus a fee plus a profit. 

In [20]:
def run(triton_url, prompt):

        payload = {
            "text_input": prompt,
            "max_tokens": 1500,
            "temperature": 0.0,
            "stream": False
        }    

        ret = requests.post(
            triton_url,
            json=payload,
        )
        res = ret.json()
        query_response = res["text_output"]
        return query_response
run("http://172.22.214.120:28800/v2/models/llama-3.1-8b-instruct/generate", "HI")

"HIKING IN THE ALPS\nThe Alps are a hiker's paradise, with towering peaks, picturesque villages, and breathtaking scenery. From easy day hikes to multi-day treks, there's something for every level of hiker. Here are some of the best hiking trails in the Alps:\n1. Haute Route, Switzerland: This 7-day trek takes you through some of the most beautiful scenery in the Alps, including glaciers, mountain lakes, and picturesque villages.\n2. Tour du Mont Blanc, France/Italy/Switzerland: This 7-day trek takes you around the highest peak in the Alps, with stunning views of glaciers, mountain peaks, and picturesque villages.\n3. Tour of the Matterhorn, Switzerland: This 4-day trek takes you around the iconic Matterhorn peak, with breathtaking views of glaciers, mountain peaks, and picturesque villages.\n4. The Tour of the Jungfrau Region, Switzerland: This 4-day trek takes you through the picturesque villages and stunning scenery of the Jungfrau Region, including the famous Jungfraujoch mountain.

In [81]:
run("http://172.22.214.120:28800/v2/models/llama-3.1-8b-instruct/generate", "Hi!")

"Hi! I'm interested in learning more about the different types of trees that are native to the Pacific Northwest region of North America. The Pacific Northwest (PNW) region includes parts of Washington, Oregon, and British Columbia, Canada. This region is known for its lush forests, mild climate, and diverse tree species. Here are some of the most common types of trees found in the PNW:\n\n1.  **Western Red Cedar (Thuja plicata)**: This coniferous tree is native to the PNW and is known for its distinctive, aromatic wood and its ability to thrive in wet environments.\n2.  **Douglas Fir (Pseudotsuga menziesii)**: A popular Christmas tree species, the Douglas Fir is also a common sight in PNW forests. It's known for its soft needles and conical shape.\n3.  **Western Hemlock (Tsuga heterophylla)**: This evergreen tree is a staple of PNW forests, with its soft needles and conical shape. It's also a popular choice for Christmas trees.\n4.  **Bigleaf Maple (Acer macrophyllum)**: This deciduou

In [84]:
run("http://172.22.214.120:28800/v2/models/llama-3.2-3b-instruct/generate", "To answer the following question, is some arithmetic calculation required on data? QUESTION: What is the company paid on a cost-plus type contract?  Please answer with one word: YES or NO. Nothing else." )

'To answer the following question, is some arithmetic calculation required on data? QUESTION: What is the company paid on a cost-plus type contract?  Please answer with one word: YES or NO. Nothing else. \nAnswer: YES'

In [86]:
system_prompt = "You are an expert on categorizing questions. You receive a context as TEXT, a TABLE and a QUESTION. You have to categorise question, into two categories: EXTRACTED and CALCULATED. EXTRACTED means that the answer to the question can be extracted without calcuation. CALCULATED means that the answer to the question can be calculated from the context. Your response MUST contain only one word, either EXTRACTED, or CALCULATED."
query = f"Given the following TEXT: {text} and TABLE: {table}, categorize the following QUESTION: {question}"
prompt = f"<|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

In [87]:
run("http://172.22.214.120:28800/v2/models/llama-3.2-3b-instruct/generate", prompt)

"<|start_header_id|>system<|end_header_id|>You are an expert on categorizing questions. You receive a context as TEXT, a TABLE and a QUESTION. You have to categorise question, into two categories: EXTRACTED and CALCULATED. EXTRACTED means that the answer to the question can be extracted without calcuation. CALCULATED means that the answer to the question can be calculated from the context. Your response MUST contain only one word, either EXTRACTED, or CALCULATED.<|eot_id|><|start_header_id|>user<|end_header_id|>Given the following TEXT: NOTE 13 - TAXES ON INCOME;B. Deferred income taxes:;Deferred income taxes reflect the net tax effects of temporary differences between the carrying amounts of assets and liabilities for financial reporting purposes and the amounts used for income tax purposes. Significant components of the Company's deferred tax assets are as follows:;As of December 31, 2019, the Company has provided a full valuation allowances of $19,911 in respect of deferred tax asse

In [21]:
def eval_res2(res):
    good_type = 0
    for i in res:            
        if (i["raw"]["answer_type"] == "arithmetic" and  i["type_response"] == "CALCULATED") or (i["raw"]["answer_type"] != "arithmetic" and  i["type_response"] == "EXTRACTED"):
            good_type = good_type+1
        else:
            print("**", end="")        
            print(i["raw"]["answer_type"],"\t",i["type_response"],"\t", i["question"])
   
    print(good_type, len(res), good_type / len(res))

In [24]:
system_prompt = "You are an expert on categorizing questions. You receive a context as TEXT, a TABLE and a QUESTION. You have to categorise question, into two categories: EXTRACTED and CALCULATED. EXTRACTED means that the answer to the question can be extracted without calcuation. CALCULATED means that the answer to the question can be calculated from the context. Your response MUST contain only one word, either EXTRACTED, or CALCULATED. You ARE NOT ALLOWED to write ANY other words."

os.environ["TRITON_LLM_ENDPOINT"] =  "http://172.22.214.120:28800/v2/models/llama-3.1-8b-instruct/generate"
res = []
llm = TritonLLM2()
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        query = f"Given the following TEXT: {text} and TABLE: {table}, categorize the following QUESTION: {question}"
        prompt = f"<|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
        #answer = llm.invoke(prompt).replace(prompt, "").strip()
        response = run("http://172.22.214.120:28800/v2/models/llama-3.1-8b-instruct/generate", prompt).replace(prompt, "").strip()
        res.append({'raw': q, "question":question,  "type_prompt": prompt, "type_response": response})
eval_res2(res)

**span 	 CALCULATED 	 What is the company paid on a cost-plus type contract?
**span 	 CALCULATED 	 What is the amount of total sales in 2019?
**multi-span 	 (EXTRACTED) 	 What are the contract types?
**span 	 CALCULATED 	 In which year is the amount of total sales the largest?
**span 	 (EXTRACTED) 	 How is industry end market information presented?
**multi-span 	 CALCULATED 	 In which years was for the net sales by segment and industry end market calculated?
**multi-span 	 (EXTRACTED) 	 What are the types of Solutions segments in the table?
**span 	 CALCULATED 	 In which year was the amount for Sensors the largest?
**span 	 (EXTRACTED) 	 How is the discount rate for domestic plans determined?
**span 	 CALCULATED 	 How is the discount rate for international plans determined?
**span 	 CALCULATED 	 How often does the company review the actuarial assumptions which the periodic benefit cost and the actuarial present value of projected benefit obligations are based on?
**multi-span 	 CALCULA

In [25]:
import time 
system_prompt = "You are an expert on categorizing questions. You receive a context as TEXT, a TABLE and a QUESTION. You have to categorise question, into two categories: EXTRACTED and CALCULATED. EXTRACTED means that the answer to the question can be extracted without calcuation. CALCULATED means that the answer to the question can be calculated from the context. Your response MUST contain only one word, either EXTRACTED, or CALCULATED. You ARE NOT ALLOWED to write ANY other words."

os.environ["TRITON_LLM_ENDPOINT"] =  "http://172.22.214.120:28800/v2/models/llama-3.1-8b-instruct/generate"
res = []
llm = TritonLLM2()
etime = 0
call = 0
for idx,row in df.iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        query = f"Given the following TEXT: {text} and TABLE: {table}, categorize the following QUESTION: {question}"
        prompt = f"<|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
        #answer = llm.invoke(prompt).replace(prompt, "").strip()
        start = time.time()
        response = run("http://172.22.214.120:28800/v2/models/llama-3.1-8b-instruct/generate", prompt).replace(prompt, "").strip()
        end = time.time()
        etime = etime + (end - start)
        res.append({'raw': q, "question":question,  "type_prompt": prompt, "type_response": response})
        call = call + 1
eval_res2(res)
print (etime/call)

**span 	 CALCULATED 	 What is the company paid on a cost-plus type contract?
**span 	 CALCULATED 	 What is the amount of total sales in 2019?
**multi-span 	 (EXTRACTED) 	 What are the contract types?
**span 	 CALCULATED 	 In which year is the amount of total sales the largest?
**span 	 (EXTRACTED) 	 How is industry end market information presented?
**multi-span 	 CALCULATED 	 In which years was for the net sales by segment and industry end market calculated?
**multi-span 	 (EXTRACTED) 	 What are the types of Solutions segments in the table?
**span 	 CALCULATED 	 In which year was the amount for Sensors the largest?
**span 	 (EXTRACTED) 	 How is the discount rate for domestic plans determined?
**span 	 CALCULATED 	 How is the discount rate for international plans determined?
**span 	 CALCULATED 	 How often does the company review the actuarial assumptions which the periodic benefit cost and the actuarial present value of projected benefit obligations are based on?
**multi-span 	 CALCULA

In [26]:
print (etime/call, etime,call)


0.10139979875916771 169.13486433029175 1668


In [27]:
def eval_res3(res):
    good_type = 0
    good_type_len = 0

    bad_type = 0
    bad_type_len = 0
    
    for i in res:            
        if (i["raw"]["answer_type"] == "arithmetic" and  i["type_response"] == "CALCULATED") or (i["raw"]["answer_type"] != "arithmetic" and  i["type_response"] == "EXTRACTED"):
            good_type = good_type+1
            good_type_len = good_type_len + len(i["type_prompt"])
        else:
            print("**", end="")        
            print(i["raw"]["answer_type"],"\t",len(i["type_prompt"]), "\t", i["type_response"],"\t", i["question"])

            bad_type = bad_type+1
            bad_type_len = bad_type_len + len(i["type_prompt"])

   
    print(good_type, len(res), good_type / len(res), good_type_len/good_type , bad_type_len/bad_type )
eval_res3(res)

**span 	 1826 	 CALCULATED 	 What is the company paid on a cost-plus type contract?
**span 	 1814 	 CALCULATED 	 What is the amount of total sales in 2019?
**multi-span 	 1800 	 (EXTRACTED) 	 What are the contract types?
**span 	 1827 	 CALCULATED 	 In which year is the amount of total sales the largest?
**span 	 1784 	 (EXTRACTED) 	 How is industry end market information presented?
**multi-span 	 1818 	 CALCULATED 	 In which years was for the net sales by segment and industry end market calculated?
**multi-span 	 1789 	 (EXTRACTED) 	 What are the types of Solutions segments in the table?
**span 	 1788 	 CALCULATED 	 In which year was the amount for Sensors the largest?
**span 	 1948 	 (EXTRACTED) 	 How is the discount rate for domestic plans determined?
**span 	 1953 	 CALCULATED 	 How is the discount rate for international plans determined?
**span 	 2063 	 CALCULATED 	 How often does the company review the actuarial assumptions which the periodic benefit cost and the actuarial presen